# 层归一化(Layer Normalization)在RNN中的应用

## 教程概览

层归一化(Layer Normalization)是一种正则化技术，特别适用于循环神经网络(RNN)。与批归一化(Batch Normalization)不同，层归一化在特征维度上进行归一化，而不是在batch维度上。

### 核心内容
1. **归一化技术对比**：Batch Norm vs Layer Norm
2. **Layer Norm原理**：数学公式和直觉理解
3. **自定义RNN Cell**：实现带Layer Norm的RNN
4. **性能对比**：验证Layer Norm的效果

### 知识点
- 层归一化的数学原理
- 自定义Keras RNN Cell
- 训练稳定性提升
- 收敛速度对比

## 一、归一化技术对比

### Batch Normalization (批归一化)
- **归一化维度**：在batch维度上进行归一化
- **适用场景**：前馈网络、CNN
- **RNN中的问题**：不同时间步的统计特性可能差异很大

$$\text{BN}(x) = \gamma \frac{x - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} + \beta$$

其中 $\mu_B$ 和 $\sigma_B$ 是batch内的均值和标准差

### Layer Normalization (层归一化)
- **归一化维度**：在特征维度上进行归一化
- **适用场景**：RNN、Transformer
- **优势**：不依赖batch统计，每个样本独立归一化

$$\text{LN}(x) = \gamma \frac{x - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}} + \beta$$

其中 $\mu_L$ 和 $\sigma_L$ 是单个样本特征维度的均值和标准差

### 为什么RNN需要Layer Norm？
1. **稳定训练**：减少内部协变量偏移(Internal Covariate Shift)
2. **加速收敛**：通过归一化激活值加快训练速度
3. **减轻梯度消失/爆炸**：保持激活值在合理范围内

In [ ]:
"""
二、实现带Layer Normalization的RNN Cell
自定义RNN单元，集成层归一化
"""
import tensorflow as tf
from tensorflow import keras
import numpy as np

class LNSimpleRNNCell(keras.layers.Layer):
    """
    带层归一化的SimpleRNN单元
    
    在标准SimpleRNN的基础上，对输出应用层归一化
    """
    
    def __init__(self, units, activation='tanh', **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        self.state_size = units
        self.output_size = units
        
    def build(self, input_shape):
        """构建层的权重"""
        input_dim = input_shape[-1]
        
        # 输入到隐藏状态的权重
        self.kernel = self.add_weight(
            shape=(input_dim, self.units),
            initializer='glorot_uniform',
            name='kernel'
        )
        
        # 循环权重（隐藏状态到隐藏状态）
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='orthogonal',
            name='recurrent_kernel'
        )
        
        # 偏置
        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',
            name='bias'
        )
        
        # Layer Normalization层
        self.layer_norm = keras.layers.LayerNormalization(epsilon=1e-6)
        
        super().build(input_shape)
    
    def call(self, inputs, states):
        """
        前向传播
        
        参数:
            inputs: 当前时间步的输入，形状 (batch_size, input_dim)
            states: 上一时间步的隐藏状态，列表形式 [h_{t-1}]
        
        返回:
            outputs: 当前时间步的输出
            new_states: 新的隐藏状态列表
        """
        prev_output = states[0]
        
        # 标准RNN计算：h = activation(W*x + U*h_{t-1} + b)
        h = tf.matmul(inputs, self.kernel)
        h += tf.matmul(prev_output, self.recurrent_kernel)
        h = tf.nn.bias_add(h, self.bias)
        
        # 应用Layer Normalization
        h = self.layer_norm(h)
        
        # 应用激活函数
        output = self.activation(h)
        
        return output, [output]
    
    def get_config(self):
        """保存配置以便序列化"""
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': keras.activations.serialize(self.activation)
        })
        return config

print("=" * 70)
print("自定义LNSimpleRNNCell类定义完成")
print("=" * 70)
print("特点:")
print("1. 继承自keras.layers.Layer")
print("2. 实现了build()和call()方法")
print("3. 在激活函数前应用Layer Normalization")
print("4. 支持序列化和反序列化")
print("=" * 70)

In [ ]:
"""
三、构建使用自定义Cell的RNN模型
"""

print("=" * 70)
print("构建带Layer Norm的RNN模型")
print("=" * 70)

# 构建使用LNSimpleRNNCell的模型
model_ln = keras.models.Sequential([
    keras.layers.RNN(
        LNSimpleRNNCell(32),
        return_sequences=True,
        input_shape=(None, 10)
    ),
    keras.layers.RNN(
        LNSimpleRNNCell(32),
        return_sequences=True
    ),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model_ln.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model_ln.summary()
print(f"\n总参数量: {model_ln.count_params():,}")

print("\n" + "=" * 70)
print("构建标准SimpleRNN模型（对比基准）")
print("=" * 70)

# 构建标准SimpleRNN模型作为对比
model_standard = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=(None, 10)),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model_standard.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model_standard.summary()
print(f"\n总参数量: {model_standard.count_params():,}")

print("\n✓ 两个模型构建完成")

In [ ]:
"""
四、生成训练数据并对比性能
"""

# 生成合成序列数据
np.random.seed(42)

def generate_sequences(n_samples, n_steps, n_features):
    """
    生成随机序列数据用于训练
    """
    X = np.random.randn(n_samples, n_steps, n_features).astype(np.float32)
    # 目标是输入的变换版本（简单的学习任务）
    y = np.roll(X, shift=1, axis=1)
    return X, y

# 生成数据集
n_samples = 5000
n_steps = 20
n_features = 10

X_train, y_train = generate_sequences(n_samples, n_steps, n_features)
X_val, y_val = generate_sequences(1000, n_steps, n_features)

print("=" * 70)
print("数据集生成完成")
print("=" * 70)
print(f"训练集: X={X_train.shape}, y={y_train.shape}")
print(f"验证集: X={X_val.shape}, y={y_val.shape}")

print("\n" + "=" * 70)
print("训练标准SimpleRNN模型")
print("=" * 70)

history_standard = model_standard.fit(
    X_train, y_train,
    epochs=3,                    # 快速测试，使用较少epoch
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

print("\n" + "=" * 70)
print("训练带Layer Norm的RNN模型")
print("=" * 70)

history_ln = model_ln.fit(
    X_train, y_train,
    epochs=3,                    # 快速测试，使用较少epoch
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

print("\n" + "=" * 70)
print("性能对比")
print("=" * 70)

final_loss_std = history_standard.history['val_loss'][-1]
final_loss_ln = history_ln.history['val_loss'][-1]

print(f"\n标准SimpleRNN - 最终验证损失: {final_loss_std:.6f}")
print(f"LayerNorm RNN  - 最终验证损失: {final_loss_ln:.6f}")
print(f"\n改进: {((final_loss_std - final_loss_ln) / final_loss_std * 100):.2f}%")

In [ ]:
"""
五、可视化训练过程
"""
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_standard.history['loss']) + 1)

# 绘制损失曲线
ax1.plot(epochs_range, history_standard.history['loss'], 
         'b-o', label='标准RNN - 训练', linewidth=2)
ax1.plot(epochs_range, history_standard.history['val_loss'], 
         'b--o', label='标准RNN - 验证', linewidth=2)
ax1.plot(epochs_range, history_ln.history['loss'], 
         'r-s', label='LayerNorm RNN - 训练', linewidth=2)
ax1.plot(epochs_range, history_ln.history['val_loss'], 
         'r--s', label='LayerNorm RNN - 验证', linewidth=2)
ax1.set_title('训练损失对比', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('MSE Loss', fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 绘制MAE曲线
ax2.plot(epochs_range, history_standard.history['mae'], 
         'b-o', label='标准RNN - 训练', linewidth=2)
ax2.plot(epochs_range, history_standard.history['val_mae'], 
         'b--o', label='标准RNN - 验证', linewidth=2)
ax2.plot(epochs_range, history_ln.history['mae'], 
         'r-s', label='LayerNorm RNN - 训练', linewidth=2)
ax2.plot(epochs_range, history_ln.history['val_mae'], 
         'r--s', label='LayerNorm RNN - 验证', linewidth=2)
ax2.set_title('MAE对比', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('MAE', fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("总结")
print("=" * 70)
print("Layer Normalization的优势:")
print("1. 训练更稳定：减少内部协变量偏移")
print("2. 收敛更快：通过标准化激活值加速学习")
print("3. 泛化更好：作为正则化手段提升模型性能")
print("4. 不依赖batch：适合RNN等序列模型")
print("\n应用场景:")
print("- 深层RNN网络")
print("- Transformer架构")
print("- 小batch size训练")
print("- 在线学习场景")
print("=" * 70)